# Car Price Regressor


In [1]:
import numpy as np

import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

In [2]:
np.random.seed(42)

In [3]:
df = pd.read_csv("./data/car-sales.csv").sample(frac=1)

df.rename(
    columns={
        "Make": "make",
        "Colour": "color",
        "Odometer (KM)": "odometer",
        "Doors": "doors",
        "Price": "price",
    },
    inplace=True,
)

df.dropna(subset=["price"], inplace=True)

In [4]:
X = df.drop("price", axis=1)
y = df["price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [5]:
preprocessor = ColumnTransformer(
    [
        (
            "preprocessor_make_color",
            Pipeline(
                [
                    ("imputer", SimpleImputer(strategy="constant", fill_value="NA")),
                    ("encoder", OneHotEncoder(handle_unknown="ignore")),
                ],
            ),
            ["make", "color"],
        ),
        (
            "preprocessor_doors",
            Pipeline(
                [
                    (
                        "imputer",
                        SimpleImputer(
                            strategy="constant", fill_value=X_train["doors"].mode()[0]
                        ),
                    ),
                    ("encoder", OneHotEncoder(handle_unknown="ignore")),
                ],
            ),
            ["doors"],
        ),
        (
            "preprocessor_odometer",
            Pipeline(
                [
                    (
                        "imputer",
                        SimpleImputer(
                            strategy="constant", fill_value=X_train["odometer"].mean()
                        ),
                    ),
                ],
            ),
            ["odometer"],
        ),
    ],
    remainder="passthrough",
)

estimator = RandomForestRegressor()

In [6]:
model = GridSearchCV(
    Pipeline(
        [
            ("preprocessor", preprocessor),
            ("estimator", estimator),
        ],
    ),
    {
        "estimator__n_estimators": [100, 1000],
        "estimator__min_samples_split": [2, 4],
        "estimator__max_depth": [None, 5],
        "estimator__max_features": [1, 0.5],
    },
    cv=5,
)

model.fit(X_train, y_train)

model.score(X_test, y_test)

TypeError: GridSearchCV.__init__() missing 1 required positional argument: 'param_grid'